In [2]:
import pandas as pd
import time
import re

In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [16]:
108/6

18.0

In [4]:
# driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))
driver = webdriver.Chrome()

url = "https://www.airbnb.com/s/Thessaloniki--Greece/homes?tab_id=home_tab&flexible_trip_lengths%5B%5D=one_week&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJ7eAoFPQ4qBQRqXTVuBXnugk&query=Thessaloniki%2C%20Greece&date_picker_type=calendar&search_type=user_map_move&price_filter_input_type=0&ne_lat=40.66256734970964&ne_lng=23.003752862853986&sw_lat=40.59051931897441&sw_lng=22.892087137145978&zoom=13&search_by_map=true&federated_search_session_id=1ed21e1c-0c5e-4529-ab84-267361eac02b&pagination_search=true&items_offset={offset}&section_offset=2"

data = []
for offset in range(0, 120, 20):
    driver.get(url.format(offset = offset))
    driver.maximize_window()
    time.sleep(2)
    # soup1 = BeautifulSoup(driver.page_source, "lxml")
    soup1 = BeautifulSoup(driver.page_source, "html.parser")

    detailed_pages = []
    for i in soup1.findAll("meta", {"itemprop": "url"}):
        detailed_pages.append(i.get("content"))

    for page in detailed_pages:
        driver.get(f"https://{page}")
        time.sleep(2)
        # soup2 = BeautifulSoup(driver.page_source, "lxml")
        soup2 = BeautifulSoup(driver.page_source, "html.parser")
        rooms =  soup2.find("ol", {"class": "lgx66tx dir dir-ltr"}).get_text() if soup2.find("ol", {"class": "lgx66tx dir dir-ltr"}) else None
        price = soup2.find("span", {"class": "_tyxjp1"}).get_text() if soup2.find("span", {"class": "_tyxjp1"}) else None
        rating = soup2.find("span",{"class":"_17p6nbba"}).get_text() if soup2.find("span",{"class":"_17p6nbba"}) else None
        location = soup2.find("span",{"class":"_9xiloll"}).get_text() if soup2.find("span",{"class":"_9xiloll"}) else None

        # host details
        # host_details = []
        # host_features = soup2.findAll("span", {"class": "l1dfad8f dir dir-ltr"}) if soup2.findAll("span",{"class":"l1dfad8f dir dir-ltr"}) else []
        # from pdb import set_trace
        # set_trace()
        # for host_feature in host_features:
        #     # print(host_feature.get_text())
        #     host_details.append(host_feature.get_text())

        # amenities
        driver.get(f"https://{page.split('?')[0]}/amenities?{page.split('?')[1]}")
        time.sleep(3)
        # soup3 = BeautifulSoup(driver.page_source, "lxml")
        soup3 = BeautifulSoup(driver.page_source, "html.parser")
        amenities = soup3.findAll("div", {"class": "twad414 dir dir-ltr"}) if soup3.findAll("div", {"class": "twad414 dir dir-ltr"}) else []
        place_offers = []
        for amenity in amenities:
            place_offers.append(amenity.get_text())

        # reviews
        driver.get(f"https://{page.split('?')[0]}/reviews?{page.split('?')[1]}")
        time.sleep(2)
        reviews_soup = BeautifulSoup(driver.page_source, "html.parser")
        reviews = reviews_soup.findAll("div", {"class": "_yorrb7h"}) if reviews_soup.findAll("div", {"class": "_yorrb7h"}) else []
        detail_ratings = []
        for review in reviews:
            detail_ratings.append(review.get_text())

        data.append({"price": price,
                     "location":location,
                     "rooms": rooms,
                     "rating":rating,
                    #  "host_details":host_details,
                     "detail_ratings":detail_ratings,
                     "place_offers": place_offers})




In [9]:
len(data)

108

In [10]:
df = pd.DataFrame(data)

In [11]:
df.head()

,price,location,rooms,rating,detail_ratings,place_offers
0,€ 117,"Agios Pavlos, Greece",3 guests · · 1 bedroom · · 1 bed · · 1 bath,5.0 ·,"[Cleanliness5.0, Accuracy5.0, Communication5.0...","[Bathtub, Hair dryer, Cleaning products, Body ..."
1,€ 100,"Thessaloniki, Greece",5 guests · · 2 bedrooms · · 3 beds · · 1 bath,4.97 ·,"[Cleanliness5.0, Accuracy5.0, Communication4.9...","[City skyline view, Hair dryer, Free washer – ..."
2,None,"Thessaloniki, Greece",4 guests · · 1 bedroom · · 2 beds · · 1 bath,5.0 ·,"[Cleanliness5.0, Accuracy5.0, Communication5.0...","[Hair dryer, Shampoo, Hot water, Shower gel, W..."
3,€ 150,"Thessaloniki, Greece",4 guests · · 2 bedrooms · · 2 beds · · 2.5 ...,4.94 ·,"[Cleanliness4.9, Accuracy4.9, Communication5.0...","[Mountain view, City skyline view, Sea view, H..."
4,€ 203,"Thessaloniki, Greece",4 guests · · 1 bedroom · · 2 beds · · 1 bath,None,[],"[Hair dryer, LUX shampoo, LUX body soap, Hot w..."


In [12]:
df.to_csv('airbnb_greece.csv',index=False)

In [13]:
df.isna().sum()

price             24
location           0
rooms              0
rating            14
detail_ratings     0
place_offers       0
dtype: int64

https://www.airbnb.com/s/Thessaloniki--Greece/homes?tab_id=home_tab&flexible_trip_lengths%5B%5D=one_week&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJ7eAoFPQ4qBQRqXTVuBXnugk&query=Thessaloniki%2C%20Greece&date_picker_type=calendar&search_type=user_map_move&price_filter_input_type=0&ne_lat=40.66256734970964&ne_lng=23.003752862853986&sw_lat=40.59051931897441&sw_lng=22.892087137145978&zoom=13&search_by_map=true&monthly_start_date=2023-09-01&monthly_length=3&price_filter_num_nights=5&channel=EXPLORE&federated_search_session_id=1ed21e1c-0c5e-4529-ab84-267361eac02b&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MCwiaXRlbXNfb2Zmc2V0IjowLCJ2ZXJzaW9uIjoxfQ%3D%3D

https://www.airbnb.com/s/Thessaloniki--Greece/homes?tab_id=home_tab&flexible_trip_lengths%5B%5D=one_week&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJ7eAoFPQ4qBQRqXTVuBXnugk&query=Thessaloniki%2C%20Greece&date_picker_type=calendar&search_type=user_map_move&price_filter_input_type=0&ne_lat=40.66256734970964&ne_lng=23.003752862853986&sw_lat=40.59051931897441&sw_lng=22.892087137145978&zoom=13&search_by_map=true&monthly_start_date=2023-09-01&monthly_length=3&price_filter_num_nights=5&channel=EXPLORE&federated_search_session_id=1ed21e1c-0c5e-4529-ab84-267361eac02b&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MywiaXRlbXNfb2Zmc2V0IjoxOCwidmVyc2lvbiI6MX0%3D